In [2]:
import pandas as pd

# 1. 평점 데이터 (u.data) 불러오기
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])

# 2. 영화 정보 (u.item) 불러오기 (ID와 제목만)
items = pd.read_csv('ml-100k/u.item', sep='|', encoding='latin-1', header=None, usecols=[0, 1])
items.columns = ['movie_id', 'movie_title']

# 3. 두 데이터를 하나로 합치기 (이게 바로 우리가 쓸 'data'입니다!)
data = pd.merge(ratings, items, on='movie_id')

print(data.head()) # 이제 여기서 'movie_title'까지 포함된 데이터를 볼 수 있습니다.

   user_id  movie_id  rating  timestamp                 movie_title
0      196       242       3  881250949                Kolya (1996)
1      186       302       3  891717742    L.A. Confidential (1997)
2       22       377       1  878887116         Heavyweights (1994)
3      244        51       2  880606923  Legends of the Fall (1994)
4      166       346       1  886397596         Jackie Brown (1997)


In [7]:
#ID 인덱싱
user_ids = data['user_id'].unique().tolist()
user2idx = {x: i for i, x in enumerate(user_ids)}

movie_ids = data['movie_id'].unique().tolist()
movie2idx = {x: i for i, x in enumerate(movie_ids)}

data['user'] = data['user_id'].map(user2idx)
data['movie'] = data['movie_id'].map(movie2idx)

n_users = len(user_ids)
n_movies = len(movie_ids)

data.head()

,user_id,movie_id,rating,timestamp,movie_title,user,movie
0,196,242,3,881250949,Kolya (1996),0,0
1,186,302,3,891717742,L.A. Confidential (1997),1,1
2,22,377,1,878887116,Heavyweights (1994),2,2
3,244,51,2,880606923,Legends of the Fall (1994),3,3
4,166,346,1,886397596,Jackie Brown (1997),4,4


In [9]:
import joblib # 딕셔너리 같은 객체 저장용
import os

# 1. 전처리된 통합 데이터 저장
data.to_csv('data/processed_data.csv', index=False)

# 2. 핵심 번역기(딕셔너리) 저장 (이게 없으면 나중에 서비스 못 합니다!)
# models/ 폴더가 없다면 생성 후 저장하세요
if not os.path.exists('models'): os.makedirs('models')
joblib.dump(user2idx, 'models/user2idx.joblib')
joblib.dump(movie2idx, 'models/movie2idx.joblib')

print("데이터 및 인덱스 저장 완료!")

데이터 및 인덱스 저장 완료!


In [8]:
from sklearn.model_selection import train_test_split

# 평점(rating)을 예측하는 것이 목표이므로, X는 ID들, y는 평점으로 둡니다.
X = data[['user', 'movie']].values
y = data['rating'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"학습 데이터 개수: {len(X_train)}")
print(f"테스트 데이터 개수: {len(X_test)}")

학습 데이터 개수: 80000
테스트 데이터 개수: 20000
